<a href="https://colab.research.google.com/github/SamuelDeMarco-Dev/AtividadeProblematizadoraIA/blob/main/AtividadeProblematizadoraIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ========================================
# SimCLR - Reproduzindo parcialmente no Colab
# Pré-treino + Linear Evaluation
# ========================================


In [ ]:
!pip install torch torchvision matplotlib --quiet

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as T
from torch.utils.data import DataLoader

# ----- 1. Dataset CIFAR-10 com augmentations -----

In [ ]:
transform = T.Compose([
    T.RandomResizedCrop(32),
    T.RandomHorizontalFlip(),
    T.ColorJitter(0.4, 0.4, 0.4, 0.1),
    T.RandomGrayscale(p=0.2),
    T.ToTensor()
])

trainset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=256, shuffle=True, num_workers=2)

test_transform = T.ToTensor()
testset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=test_transform)
testloader = DataLoader(testset, batch_size=256, shuffle=False, num_workers=2)


100%|██████████| 170M/170M [00:04<00:00, 34.8MB/s]


# ----- 2. Encoder backbone (ResNet18 simplificada) -----

In [ ]:
from torchvision.models import resnet18

class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        backbone = resnet18(weights=None)
        self.features = nn.Sequential(*list(backbone.children())[:-1])  # até penúltima camada
        self.fc = nn.Linear(512, 128)  # projeção SimCLR
    def forward(self, x):
        h = self.features(x)
        h = torch.flatten(h, 1)
        z = self.fc(h)
        return F.normalize(z, dim=-1)

# Check for CUDA availability and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder = Encoder().to(device)

# ----- 3. Loss de SimCLR (NT-Xent) -----

In [ ]:
def simclr_loss(features, temperature=0.5):
    N = features.shape[0] // 2
    labels = torch.cat([torch.arange(N) for _ in range(2)], dim=0).to(features.device)
    sim = torch.matmul(features, features.T) / temperature
    mask = torch.eye(2*N, dtype=torch.bool).to(features.device)
    sim = sim.masked_fill(mask, -9e15)
    loss = F.cross_entropy(sim, labels)
    return loss

# ----- 4. Treino reduzido -----

In [ ]:
opt = torch.optim.Adam(encoder.parameters(), lr=3e-4)

for epoch in range(2):  # poucas épocas para teste
    for imgs, _ in trainloader:
        # Duas views do mesmo batch
        x1, x2 = imgs, imgs
        x1, x2 = x1.to(device), x2.to(device)
        f1, f2 = encoder(x1), encoder(x2)
        features = torch.cat([f1, f2], dim=0)
        loss = simclr_loss(features)
        opt.zero_grad()
        loss.backward()
        opt.step()
    print(f"Epoch {epoch+1} - Loss: {loss.item():.4f}")

Epoch 1 - Loss: 4499999907905536.0000
Epoch 2 - Loss: 4499999907905536.0000


# ----- 5. Linear evaluation -----
# Congelar encoder e treinar classificador linear simples

In [ ]:
class LinearEval(nn.Module):
    def __init__(self, encoder, num_classes=10):
        super().__init__()
        self.encoder = encoder
        for p in self.encoder.parameters():
            p.requires_grad = False
        self.fc = nn.Linear(128, num_classes)
    def forward(self, x):
        z = self.encoder(x)
        return self.fc(z)

linear_model = LinearEval(encoder).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(linear_model.fc.parameters(), lr=1e-3)

# Treino rápido do classificador linear

In [ ]:
for epoch in range(5):
    linear_model.train()
    total_loss, correct, total = 0, 0, 0
    for imgs, labels in trainloader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = linear_model(imgs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)
    acc = 100. * correct / total
    print(f"Epoch {epoch+1} - Loss: {total_loss/len(trainloader):.4f}, Train Acc: {acc:.2f}%")

Epoch 1 - Loss: 2.2910, Train Acc: 13.89%
Epoch 2 - Loss: 2.2673, Train Acc: 18.31%
Epoch 3 - Loss: 2.2478, Train Acc: 19.58%
Epoch 4 - Loss: 2.2348, Train Acc: 19.57%
Epoch 5 - Loss: 2.2218, Train Acc: 19.77%


# Avaliação no conjunto de teste

In [ ]:
linear_model.eval()
correct, total = 0, 0
with torch.no_grad():
    for imgs, labels in testloader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = linear_model(imgs)
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)

test_acc = 100. * correct / total
print(f"Test Accuracy (linear eval): {test_acc:.2f}%")

Test Accuracy (linear eval): 25.41%
